<a href="https://colab.research.google.com/github/aaronjoseph/KB_Final/blob/master/3_a_RoBERTa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

     |████████████████████████████████| 1.8MB 7.9MB/s 
     |████████████████████████████████| 890kB 34.6MB/s 
     |████████████████████████████████| 2.9MB 36.5MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=54710d9dac903b8a735b6ad38941d6aa5a89df59aae374dd13f89f2248f6d895
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import torch
import seaborn as sns
import transformers
import json
from tqdm import tqdm 
from torch.utils.data import Dataset, DataLoader
from transformers import RobertaModel, RobertaTokenizer
import logging
logging.basicConfig(level=logging.ERROR)

In [3]:
# Setting device as GPU
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
print(device)
# CUDA is being used for traning

cuda


In [4]:
# Data Loading 
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/training.1600000.processed.noemoticon.csv',header=None,error_bad_lines=False,engine='python')
df.columns=['Sentiment', 'id', 'Date', 'Query', 'User', 'Phrase']
df = df.drop(columns=['id', 'Date', 'Query', 'User'], axis=1)
df['Sentiment'] = df.Sentiment.replace(4,1)

In [22]:
df_1 = df[['Phrase', 'Sentiment']]
df_new = df_1.sample(frac=0.1,random_state=200).reset_index(drop=True)

<a id='section03'></a>
### Preparing the Dataset and Dataloader

- Defining few key variables | Which will be used in the training/fine tuning stage
- Dataset Class - Will pre-process the data before feeding it into the Neural Netork
 [Pytorch Documentation on Datasets](https://pytorch.org/docs/stable/data.html)

#### *SentimentData* Dataset Class

- This class is defined to accept the Dataframe as input and generate tokenized output that is used by the Roberta model for training
- I am using the Roberta tokenizer to tokenize the data in the `TITLE` column of the dataframe. 
- The tokenizer uses the `encode_plus` method to perform tokenization and generate the necessary outputs, namely: `ids`, `attention_mask`
- To read further into the tokenizer, [refer to this document](https://huggingface.co/transformers/model_doc/roberta.html#robertatokenizer)
- `target` is the encoded category on the news headline. 
- The *SentimentData* class is used to create 2 datasets, for training and for validation.
- *Training Dataset* is used to fine tune the model: **80% of the original data**
- *Validation Dataset* is used to evaluate the performance of the model. The model has not seen this data during training. 

#### Dataloader

- Dataloader is used to for creating training and validation dataloader that load data to the neural network in a defined manner. This is needed because all the data from the dataset cannot be loaded to the memory at once, hence the amount of dataloaded to the memory and then passed to the neural network needs to be controlled.
- This control is achieved using the parameters such as `batch_size` and `max_len`.
- Training and Validation dataloaders are used in the training and validation part of the flow respectively

In [23]:
# Key Variable Defination
MAX_LEN = 256
TRAIN_BATCH_SIZE = 8
VALID_BATCH_SIZE = 4
# EPOCHS = 1
LEARNING_RATE = 1e-05
tokenizer = RobertaTokenizer.from_pretrained('roberta-base', truncation=True, do_lower_case=True)

In [24]:
class SentimentData(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.text = dataframe.Phrase
        self.targets = self.data.Sentiment
        self.max_len = max_len

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        text = str(self.text[index])
        text = " ".join(text.split())

        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]


        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'targets': torch.tensor(self.targets[index], dtype=torch.float)
        }

In [25]:
new_df

,Phrase,Sentiment
0,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",0
1,is upset that he can't update his Facebook by ...,0
2,@Kenichan I dived many times for the ball. Man...,0
3,my whole body feels itchy and like its on fire,0
4,"@nationwideclass no, it's not behaving at all....",0
...,...,...
1599995,Just woke up. Having no school is the best fee...,1
1599996,TheWDB.com - Very cool to hear old Walt interv...,1
1599997,Are you ready for your MoJo Makeover? Ask me f...,1
1599998,Happy 38th Birthday to my boo of alll time!!! ...,1


In [26]:
train_size = 0.7
train_data=new_df.sample(frac=train_size,random_state=200)
test_data=new_df.drop(train_data.index).reset_index(drop=True)
train_data = train_data.reset_index(drop=True)

print("FULL Dataset: {}".format(new_df.shape))
print("TRAIN Dataset: {}".format(train_data.shape))
print("TEST Dataset: {}".format(test_data.shape))

FULL Dataset: (1600000, 2)
TRAIN Dataset: (1120000, 2)
TEST Dataset: (480000, 2)


In [27]:
training_set = SentimentData(train_data, tokenizer, MAX_LEN)
testing_set = SentimentData(test_data, tokenizer, MAX_LEN)

train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

### Creating the Neural Network for Fine Tuning

#### Neural Network
 - We will be creating a neural network with the `RobertaClass`. 
 - This network will have the Roberta Language model followed by a `dropout` and finally a `Linear` layer to obtain the final outputs. 
 - The data will be fed to the Roberta Language model as defined in the dataset. 
 - Final layer outputs is what will be compared to the `Sentiment category` to determine the accuracy of models prediction. 
 - We will initiate an instance of the network called `model`. This instance will be used for training and then to save the final trained model for future inference. 
 
#### Loss Function and Optimizer
 - `Loss Function` and `Optimizer` and defined in the next cell.
 - The `Loss Function` is used the calculate the difference in the output created by the model and the actual output. 
 - `Optimizer` is used to update the weights of the neural network to improve its performance.

In [28]:
class RobertaClass(torch.nn.Module):
    def __init__(self):
        super(RobertaClass, self).__init__()
        self.l1 = RobertaModel.from_pretrained("roberta-base")
        self.pre_classifier = torch.nn.Linear(768, 768)
        self.dropout = torch.nn.Dropout(0.3)
        self.classifier = torch.nn.Linear(768, 2)

    def forward(self, input_ids, attention_mask, token_type_ids):
        output_1 = self.l1(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        hidden_state = output_1[0]
        pooler = hidden_state[:, 0]
        pooler = self.pre_classifier(pooler)
        pooler = torch.nn.ReLU()(pooler)
        pooler = self.dropout(pooler)
        output = self.classifier(pooler)
        return output

In [29]:
model = RobertaClass()
model.to(device)

RobertaClass(
  (l1): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), e

In [30]:
model_parameters = filter(lambda p: p.requires_grad, model.parameters())
params = sum([np.prod(p.size()) for p in model_parameters])
print("Number of Trainable Parameters = ",params)

Number of Trainable Parameters =  125237762


<a id='section05'></a>
### Fine Tuning the Model

- Here the training function of training the model takes place. Also, as defined by EPOCHs
- The dataloader passes data to the model based on the batch size. 
- Subsequent output from the model and the actual category are compared to calculate the loss. 
- Loss value is used to optimize the weights of the neurons in the network.
- After every 5000 steps the loss value is printed in the console.

In [31]:
#Loss Function & Optimizer
loss_function = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

def calcuate_accuracy(preds, targets):
    n_correct = (preds==targets).sum().item()
    return n_correct

In [32]:
def train(epoch):
    tr_loss = 0
    n_correct = 0
    nb_tr_steps = 0
    nb_tr_examples = 0
    model.train()
    for _,data in tqdm(enumerate(training_loader, 0)):
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.long)

        outputs = model(ids, mask, token_type_ids)
        loss = loss_function(outputs, targets)
        tr_loss += loss.item()
        big_val, big_idx = torch.max(outputs.data, dim=1)
        n_correct += calcuate_accuracy(big_idx, targets)

        nb_tr_steps += 1
        nb_tr_examples+=targets.size(0)
        
        if _%5000==0:
            loss_step = tr_loss/nb_tr_steps
            accu_step = (n_correct*100)/nb_tr_examples 
            print(f"Training Loss per 5000 steps: {loss_step}")
            print(f"Training Accuracy per 5000 steps: {accu_step}")

        optimizer.zero_grad()
        loss.backward()
        # # When using GPU
        optimizer.step()

    print(f'The Total Accuracy for Epoch {epoch}: {(n_correct*100)/nb_tr_examples}')
    epoch_loss = tr_loss/nb_tr_steps
    epoch_accu = (n_correct*100)/nb_tr_examples
    print(f"Training Loss Epoch: {epoch_loss}")
    print(f"Training Accuracy Epoch: {epoch_accu}")

    return 

In [ ]:
# Keeping the model at 1 EPOCH
EPOCHS = 1
for epoch in range(EPOCHS):
    train(epoch)


0it [00:00, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:2143: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,

1it [00:00,  3.02it/s]

Training Loss per 5000 steps: 0.7032142877578735
Training Accuracy per 5000 steps: 25.0


Streaming output truncated to the last 5000 lines.
2it [00:00,  3.26it/s]
3it [00:00,  3.45it/s]
4it [00:01,  3.60it/s]
5it [00:01,  3.71it/s]
6it [00:01,  3.73it/s]
7it [00:01,  3.79it/s]
8it [00:02,  3.87it/s]
9it [00:02,  3.86it/s]
10it [00:02,  3.90it/s]
11it [00:02,  3.93it/s]
12it [00:03,  3.96it/s]
13it [00:03,  3.98it/s]
14it [00:03,  3.99it/s]
15it [00:03,  3.91it/s]
16it [00:04,  3.94it/s]
17it [00:04,  3.89it/s]
18it [00:04,  3.87it/s]
19it [00:04,  3.89it/s]
20it [00:05,  3.93it/s]
21it [00:05,  3.96it/s]
22it [00:05,  3.97it/s]
23it [00:05,  3.98it/s]
24it [00:06,  4.00it/s]
25it [00:06,  4.00it/s]
26it [00:06,  4.02it/s]
27it [00:06,  4.01it/s]
28it [00:07,  3.99it/s]
29it [00:07,  3.93it/s]
30it [00:07,  3.89it/s]
31it [00:07,  3.88it/s]
32it [00:08,  3.92it/s]
33it [00:08,  3.97it/s]
34it [00:08,  3.99it/s]
35it [00:08,  3.99it/s]
36it [00:09,  3.95it/s]
37it [00:09,  3.99it/s]
38it [00:09,  4.00it/s]
39it [00:09,  3.96it/s]
40it [00:10,  3.92it/s]
41it [00:10,  3.89it/

Training Loss per 5000 steps: 0.40826819931957525
Training Accuracy per 5000 steps: 81.62367526494701


Streaming output truncated to the last 5000 lines.
5002it [21:00,  3.94it/s]
5003it [21:00,  3.98it/s]
5004it [21:00,  4.01it/s]
5005it [21:00,  4.03it/s]
5006it [21:01,  4.04it/s]
5007it [21:01,  4.05it/s]
5008it [21:01,  4.06it/s]
5009it [21:01,  4.05it/s]
5010it [21:02,  3.98it/s]
5011it [21:02,  4.00it/s]
5012it [21:02,  4.01it/s]
5013it [21:02,  4.00it/s]
5014it [21:03,  3.99it/s]
5015it [21:03,  3.90it/s]
5016it [21:03,  3.88it/s]
5017it [21:03,  3.91it/s]
5018it [21:04,  3.88it/s]
5019it [21:04,  3.89it/s]
5020it [21:04,  3.93it/s]
5021it [21:04,  3.96it/s]
5022it [21:05,  4.00it/s]
5023it [21:05,  4.01it/s]
5024it [21:05,  3.94it/s]
5025it [21:05,  3.98it/s]
5026it [21:06,  3.99it/s]
5027it [21:06,  4.02it/s]
5028it [21:06,  4.03it/s]
5029it [21:06,  4.03it/s]
5030it [21:07,  4.04it/s]
5031it [21:07,  4.05it/s]
5032it [21:07,  4.02it/s]
5033it [21:07,  3.95it/s]
5034it [21:08,  3.98it/s]
5035it [21:08,  3.99it/s]
5036it [21:08,  4.01it/s]
5037it [21:08,  4.01it/s]
5038it [21:09

...

Streaming output truncated to the last 5000 lines.
20002it [1:23:54,  3.88it/s]
20003it [1:23:54,  3.86it/s]
20004it [1:23:54,  3.91it/s]
20005it [1:23:55,  3.96it/s]
20006it [1:23:55,  3.98it/s]
20007it [1:23:55,  4.01it/s]
20008it [1:23:55,  4.02it/s]
20009it [1:23:56,  4.01it/s]
20010it [1:23:56,  4.01it/s]
20011it [1:23:56,  4.04it/s]
20012it [1:23:56,  4.04it/s]
20013it [1:23:57,  4.03it/s]
20014it [1:23:57,  4.02it/s]
20015it [1:23:57,  4.03it/s]
20016it [1:23:57,  4.05it/s]
20017it [1:23:58,  4.05it/s]
20018it [1:23:58,  4.04it/s]
20019it [1:23:58,  4.04it/s]
20020it [1:23:58,  3.98it/s]
20021it [1:23:59,  4.02it/s]
20022it [1:23:59,  4.04it/s]
20023it [1:23:59,  4.05it/s]
20024it [1:23:59,  4.06it/s]
20025it [1:23:59,  4.05it/s]
20026it [1:24:00,  3.97it/s]
20027it [1:24:00,  3.92it/s]
20028it [1:24:00,  3.88it/s]
20029it [1:24:01,  3.92it/s]
20030it [1:24:01,  3.96it/s]
20031it [1:24:01,  3.93it/s]
20032it [1:24:01,  3.91it/s]
20033it [1:24:02,  3.93it/s]
20034it [1:24:02,  3.

Training Loss per 5000 steps: 0.36616025105797323
Training Accuracy per 5000 steps: 83.97364105435783


Streaming output truncated to the last 5000 lines.
25002it [1:44:50,  3.95it/s]
25003it [1:44:50,  4.00it/s]
25004it [1:44:50,  4.02it/s]
25005it [1:44:51,  4.03it/s]
25006it [1:44:51,  4.04it/s]
25007it [1:44:51,  4.05it/s]
25008it [1:44:51,  4.00it/s]
25009it [1:44:52,  3.94it/s]
25010it [1:44:52,  3.91it/s]
25011it [1:44:52,  3.89it/s]
25012it [1:44:52,  3.87it/s]
25013it [1:44:53,  3.91it/s]
25014it [1:44:53,  3.98it/s]
25015it [1:44:53,  3.99it/s]
25016it [1:44:53,  3.96it/s]
25017it [1:44:54,  4.00it/s]
25018it [1:44:54,  4.02it/s]
25019it [1:44:54,  4.03it/s]
25020it [1:44:54,  4.04it/s]
25021it [1:44:55,  4.04it/s]
25022it [1:44:55,  4.05it/s]
25023it [1:44:55,  4.07it/s]
25024it [1:44:55,  4.08it/s]
25025it [1:44:56,  4.02it/s]
25026it [1:44:56,  4.02it/s]
25027it [1:44:56,  3.98it/s]
25028it [1:44:56,  3.98it/s]
25029it [1:44:57,  4.02it/s]
25030it [1:44:57,  4.02it/s]
25031it [1:44:57,  4.00it/s]
25032it [1:44:57,  4.02it/s]
25033it [1:44:58,  4.04it/s]
25034it [1:44:58,  3.

Training Loss per 5000 steps: 0.362117808349926
Training Accuracy per 5000 steps: 84.17677744075198



30002it [2:05:47,  3.94it/s]
30003it [2:05:47,  3.96it/s]
30004it [2:05:47,  3.97it/s]
30005it [2:05:48,  3.88it/s]
30006it [2:05:48,  3.86it/s]
30007it [2:05:48,  3.82it/s]
30008it [2:05:48,  3.86it/s]
30009it [2:05:49,  3.90it/s]
30010it [2:05:49,  3.87it/s]
30011it [2:05:49,  3.91it/s]
30012it [2:05:49,  3.88it/s]
30013it [2:05:50,  3.91it/s]
30014it [2:05:50,  3.92it/s]
30015it [2:05:50,  3.92it/s]
30016it [2:05:50,  3.84it/s]
30017it [2:05:51,  3.90it/s]
30018it [2:05:51,  3.94it/s]
30019it [2:05:51,  3.89it/s]
30020it [2:05:51,  3.86it/s]
30021it [2:05:52,  3.90it/s]
30022it [2:05:52,  3.95it/s]
30023it [2:05:52,  3.98it/s]
30024it [2:05:52,  4.01it/s]
30025it [2:05:53,  4.02it/s]
30026it [2:05:53,  4.05it/s]
30027it [2:05:53,  4.05it/s]
30028it [2:05:53,  4.00it/s]
30029it [2:05:54,  4.01it/s]
30030it [2:05:54,  4.03it/s]
30031it [2:05:54,  3.99it/s]
30032it [2:05:54,  4.01it/s]
30033it [2:05:55,  3.96it/s]
30034it [2:05:55,  3.96it/s]
30035it [2:05:55,  3.99it/s]
30036it [2:05

# Validating the Model 

Now testing the data against the 

# Saving the Trained Model Artifact for Inferencing at a later stage

In [ ]:
output_model_file = 'pytorch_roberta_sentiment.bin'
output_vocab_file = './'
model_to_save = model 
torch.save(model_to_save, output_model_file)
tokenizer.save_vocabulary(output_vocab_file)
print("Model & Tokenizer saved")